# Install & Import

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
import os
from os import walk, getcwd
import shutil
import random
import numpy as np

from sklearn.model_selection import train_test_split

from tqdm import tqdm

from glob import glob

import yaml
import json

from IPython.display import Image

from pathlib import Path
import cv2

In [ ]:
!pip install -U PyYAML

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 5.1 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install wandb

'''
이거 깃헙 올릴 때는 삭제하셈!!!!!!!!!!!
my API key :

a7ddd85e3da80c4a8503ee85e4685caa32deb381
'''

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 145 kB 80.0 MB/s 
     |████████████████████████████████| 181 kB 90.5 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=05352c6e710e503959f9dfc8a36540fb0db5f73c05a183b13f42952df3a4e2ed
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


'\n이거 깃헙 올릴 때는 삭제하셈!!!!!!!!!!!\nmy API key :\n\na7ddd85e3da80c4a8503ee85e4685caa32deb381\n'

# YOLO V5 설치

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install



Cloning into 'yolov5'...
remote: Enumerating objects: 12388, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 12388 (delta 1), reused 6 (delta 0), pack-reused 12380
Receiving objects: 100% (12388/12388), 12.16 MiB | 37.62 MiB/s, done.
Resolving deltas: 100% (8544/8544), done.
/content/yolov5


# Custom Dataset 정리

In [ ]:

## Unzip Image File
%cd /content
!unzip -qq "/content/gdrive/MyDrive/Colab Notebooks/YOLO V5/cross.zip" ## Folder 6개가 생김
print('Complete!')

/content
Complete!


In [ ]:
'''
압축이 해제된 폴더들은 Object의 수에 따라 1~6으로 구분 해둔 것임
따라서 일단 안의 파일들을 다 하나로 합쳐준 다음 Train Val 분리를 해주기로 결정
'''

os.makedirs('./dataset') ## 1~6으로 분할 되어있는 파일을 다 모아줄 폴더 생성

to_here = '/content/dataset'


for i in tqdm(range(1,7)) :
    get_file = os.listdir(f'/content/교차로정보 데이터셋_bbox_{i}') ## 폴더 내의 파일들 리스트로 묶음
    for j in get_file :
        shutil.move(f'/content/교차로정보 데이터셋_bbox_{i}/'+j, to_here) ## dataset 폴더로 모두 이동
    os.rmdir(f'/content/교차로정보 데이터셋_bbox_{i}') 

100%|██████████| 6/6 [00:02<00:00,  2.13it/s]


In [ ]:
## 파일 종류는 jpg, png // json
print(len(glob('/content/dataset/*')))
print(len(glob('/content/dataset/*.jpg')) + len(glob('/content/dataset/*.png')) + len(glob('/content/dataset/*.json'))) 

73616
73616


In [ ]:
## YOLO V5 폴더의 data 파일에 image - train/valid 폴더 생성
os.makedirs('/content/yolov5/data/images/train') 
os.makedirs('/content/yolov5/data/images/valid')

## 라벨도 형성
os.makedirs('/content/yolov5/data/labels')
os.makedirs('/content/yolov5/data/labels/train')
os.makedirs('/content/yolov5/data/labels/valid') 

img_lst = []
label_lst = []


get_file = os.listdir(f'/content/dataset') ## 폴더 내의 파일들 리스트로 묶음
for j in get_file :
    if j.endswith('.png') : 
        img_lst.append(j)
    if j.endswith('.jpg') : 
        img_lst.append(j)
    elif j.endswith('.json') :
        label_lst.append(j)

train_img , val_img = train_test_split(img_lst, test_size=0.2, random_state=42) ## train set과 validation set 분리

train_lab = []
val_lab = []

for i in tqdm(label_lst) :
    if i.split('.')[0] + '.jpg' in train_img :
        train_lab.append(i)
    elif i.split('.')[0] + '.png' in train_img :
        train_lab.append(i)

    elif i.split('.')[0] + '.jpg' in val_img :
        val_lab.append(i)

    elif i.split('.')[0] + '.png' in val_img :
        val_lab.append(i)

100%|██████████| 36808/36808 [00:27<00:00, 1353.61it/s]


In [ ]:
print(f'Train Image 수 : {len(train_img)}')
print(f'Train Label 수 : {len(train_lab)}')
print(f'Valid Image 수 : {len(val_img)}')
print(f'Valid Label 수 : {len(val_lab)}')

Train Image 수 : 29446
Train Label 수 : 29446
Valid Image 수 : 7362
Valid Label 수 : 7362


In [ ]:
for a, b in zip(train_img, train_lab) :
    shutil.move(f'/content/dataset/{a}', '/content/yolov5/data/images/train')
    shutil.move(f'/content/dataset/{b}', '/content/yolov5/data/labels/train')

for c,d in zip(val_img, val_lab) : 
    shutil.move(f'/content/dataset/{c}', '/content/yolov5/data/images/valid')
    shutil.move(f'/content/dataset/{d}', '/content/yolov5/data/labels/valid')

In [ ]:
train_path = list(sorted(glob('/content/yolov5/data/images/train/*')))
valid_path = list(sorted(glob('/content/yolov5/data/images/valid/*')))

train_label_path = list(sorted(glob('/content/yolov5/data/labels/train/*')))
valid_label_path = list(sorted(glob('/content/yolov5/data/labels/valid/*')))

In [ ]:
## 짝은 잘 형성된 모습
print(train_path[:10])
print(train_label_path[:10])
print('++++++++++')
print(valid_path[:10])
print(valid_label_path[:10])

['/content/yolov5/data/images/train/MP_KSC_000003.jpg', '/content/yolov5/data/images/train/MP_KSC_000016.jpg', '/content/yolov5/data/images/train/MP_KSC_000057.jpg', '/content/yolov5/data/images/train/MP_KSC_000058.jpg', '/content/yolov5/data/images/train/MP_KSC_000133.jpg', '/content/yolov5/data/images/train/MP_KSC_000134.jpg', '/content/yolov5/data/images/train/MP_KSC_000248.jpg', '/content/yolov5/data/images/train/MP_KSC_000249.jpg', '/content/yolov5/data/images/train/MP_KSC_000250.jpg', '/content/yolov5/data/images/train/MP_KSC_000251.jpg']
['/content/yolov5/data/labels/train/MP_KSC_000003.json', '/content/yolov5/data/labels/train/MP_KSC_000016.json', '/content/yolov5/data/labels/train/MP_KSC_000057.json', '/content/yolov5/data/labels/train/MP_KSC_000058.json', '/content/yolov5/data/labels/train/MP_KSC_000133.json', '/content/yolov5/data/labels/train/MP_KSC_000134.json', '/content/yolov5/data/labels/train/MP_KSC_000248.json', '/content/yolov5/data/labels/train/MP_KSC_000249.json', 

# Image Data Directory Define

In [ ]:
## YOLO V5 는 파일의 경로를 txt로 알려줘야 한다
with open('/content/yolov5/data/images/train.txt','w') as f :
    f.write('\n'.join(train_path) + '\n')

with open('/content/yolov5/data/images/valid.txt','w') as f :
    f.write('\n'.join(valid_path) + '\n')


# Convert Label Data from Labelme Json Format to YOLO V5 txt Format

In [ ]:
## json Label Data를 이제 txt로 바꿔줘야됨 (이게 제일 빡... ㅠ)
## class | x_center | y_center | width | height 순서임

## Train Data 변환 ##
mypath = "/content/yolov5/data/labels/train/"
outpath = "/content/yolov5/data/labels/train/"



def convert(size, box):
    dw = 1. / (size[0])
    dh = 1. / (size[1])
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

json_name_list = []
for file in os.listdir(mypath):
    if file.endswith(".json"):
        json_name_list.append(file)

## Class 정의
obj_classes = ['Zebra_Cross','G_Signal','R_Signal']


for json_file in tqdm(json_name_list) : 

    ## txt 파일로 바꿔줄 이름 !
    txt_file_path = mypath+json_file.split('.')[0] + '.txt'
    txt_label = open(txt_file_path, 'w') ## 이 파일에다가 쓸 거야!

    json_obj = json.load(open(mypath+json_file, 'r', encoding='utf-8'))  ## json 파일 오픈
    height = json_obj['imageHeight'] ## image에서 i 대문자 아니고 소문자임 ㅋㅋㅋ ㅠㅠㅠㅠㅠ
    width = json_obj['imageWidth'] ## 이미지의 높이 정보, 너비정보
    for idx, i in enumerate(json_obj['shapes']) : 
        label = i['label']
        if label == '1' : ## 중간에 라벨에 1이라고 잘못한 게 있는듯 그거 패스해버림
            continue
        points = i['points']
        if (i["shape_type"] == 'rectangle'):
            points = np.array(i["points"])
            xmin = min(points[:, 0]) if min(points[:, 0]) > 0 else 0
            xmax = max(points[:, 0]) if max(points[:, 0]) > 0 else 0
            ymin = min(points[:, 1]) if min(points[:, 1]) > 0 else 0
            ymax = max(points[:, 1]) if max(points[:, 1]) > 0 else 0

            ## 이상한 거 없나 체크
            if xmax <= xmin:
                pass
            elif ymax <= ymin:
                pass
            else:
                ## 이상 없으면 변환시킴
                bbox_labelme_float   = (float(xmin), float(xmax), float(ymin), float(ymax))
                bbox_yolo_normalized = convert((width, height), bbox_labelme_float)

                ## class_id는 obj_classes에 맞는 index가 됨
                ## 횡단보도 = 0 , 초록불 = 1 , 빨간불 =2            
                class_id = obj_classes.index(label)

                ## 드디어~~ txt 파일로 작성!!!!! (한줄)
                txt_label.write(str(class_id)+ ' ' + ' '.join([str(a) for a in bbox_yolo_normalized])+'\n')

    ## 다 썼으면 닫기
    txt_label.close()

100%|██████████| 29446/29446 [00:19<00:00, 1535.68it/s]


In [ ]:
## Valid Data 변환 ##
mypath = "/content/yolov5/data/labels/valid/"
outpath = "/content/yolov5/data/labels/valid/"


json_name_list = []
for file in os.listdir(mypath):
    if file.endswith(".json"):
        json_name_list.append(file)

## Class 정의
obj_classes = ['Zebra_Cross','G_Signal','R_Signal']


for json_file in tqdm(json_name_list) : 

    ## txt 파일로 바꿔줄 이름 !
    txt_file_path = mypath+json_file.split('.')[0] + '.txt'
    txt_label = open(txt_file_path, 'w') ## 이 파일에다가 쓸 거야!

    json_obj = json.load(open(mypath+json_file, 'r', encoding='utf-8'))  ## json 파일 오픈
    height = json_obj['imageHeight'] ## image에서 i 대문자 아니고 소문자임 ㅋㅋㅋ ㅠㅠㅠㅠㅠ
    width = json_obj['imageWidth'] ## 이미지의 높이 정보, 너비정보
    for idx, i in enumerate(json_obj['shapes']) : 
        label = i['label']
        if label not in obj_classes : ## 중간에 라벨에 1이라고 잘못한 게 있는듯 그거 패스해버림
            continue
        points = i['points']
        if (i["shape_type"] == 'rectangle'):
            points = np.array(i["points"])
            xmin = min(points[:, 0]) if min(points[:, 0]) > 0 else 0
            xmax = max(points[:, 0]) if max(points[:, 0]) > 0 else 0
            ymin = min(points[:, 1]) if min(points[:, 1]) > 0 else 0
            ymax = max(points[:, 1]) if max(points[:, 1]) > 0 else 0

            ## 이상한 거 없나 체크
            if xmax <= xmin:
                pass
            elif ymax <= ymin:
                pass
            else:
                ## 이상 없으면 변환시킴
                bbox_labelme_float   = (float(xmin), float(xmax), float(ymin), float(ymax))
                bbox_yolo_normalized = convert((width, height), bbox_labelme_float)

                ## class_id는 obj_classes에 맞는 index가 됨
                ## 횡단보도 = 0 , 초록불 = 1 , 빨간불 =2            
                class_id = obj_classes.index(label)

                ## 드디어~~ txt 파일로 작성!!!!! (한줄)
                txt_label.write(str(class_id)+ ' ' + ' '.join([str(a) for a in bbox_yolo_normalized])+'\n')

    ## 다 썼으면 닫기
    txt_label.close()


### 근데 convert 과정에서 정규화가 진행 안 된 게 몇개 보임 
### 데이터 자체의 문제인 거 같긴 함... convert 식을 다시 보기는 싫어..

100%|██████████| 7362/7362 [00:03<00:00, 1876.81it/s]


In [ ]:
print(len(list(sorted(glob('/content/yolov5/data/images/train/*')))))
print(len(list(sorted(glob('/content/yolov5/data/images/valid/*')))))

print(len(list(sorted(glob('/content/yolov5/data/labels/train/*.txt')))))
print(len(list(sorted(glob('/content/yolov5/data/labels/valid/*.txt')))))

29446
7362
29446
7362


In [ ]:
'''
이건 하나 해서 되나 안 되나 테스트 했을 때의 코드임
'''
# obj_classes = ['Zebra_Cross','G_Signal','R_Signal']


# txt_file_path = '/content/yolov5/data/labels/valid/MP_KSC_000685'+'.txt'
# txt_label = open(txt_file_path,'w')

# json_obj = json.load(open('/content/yolov5/data/labels/valid/MP_KSC_000685.json', 'r', encoding='utf-8'))  # json 파일 오픈
# height, width = json_obj['imageHeight'], json_obj['imageWidth'] ## 이미지의 높이 정보, 너비정보
# for idx, i in enumerate(json_obj['shapes']) : 
#     label = i['label']
#     points = i['points']
#     if (i["shape_type"] == 'rectangle'):
#         points = np.array(i["points"])
#         xmin = min(points[:, 0]) if min(points[:, 0]) > 0 else 0
#         xmax = max(points[:, 0]) if max(points[:, 0]) > 0 else 0
#         ymin = min(points[:, 1]) if min(points[:, 1]) > 0 else 0
#         ymax = max(points[:, 1]) if max(points[:, 1]) > 0 else 0

#         ## 이상한 거 없나 체크
#         if xmax <= xmin:
#             pass
#         elif ymax <= ymin:
#             pass
#         else:
#             ## 이상 없으면 변환시킴
#             bbox_labelme_float   = (float(xmin), float(xmax), float(ymin), float(ymax))
#             bbox_yolo_normalized = convert((float(width), float(height)), bbox_labelme_float)

#             ## class_id는 obj_classes에 맞는 index가 됨
#             ## 횡단보도 = 0 , 초록불 = 1 , 빨간불 =2            
#             class_id = obj_classes.index(label)

#             ## 드디어~~ txt 파일로 작성!!!!! (한줄)
#             txt_label.write(str(class_id)+ ' ' + ' '.join([str(a) for a in bbox_yolo_normalized])+'\n')

# ## 다 썼으면 닫기
# txt_label.close()
            

'\n이건 하나 해서 되나 안 되나 테스트 했을 때의 코드임\n'

In [ ]:

## Custom Dataset 용 yaml 파일 생성
with open('/content/yolov5/data/dataset.yaml', 'w') as f:

    f.close()


with open('/content/yolov5/data/dataset.yaml', 'r') as f:
    data = yaml.load(f) 

## 학습 데이터, 검증 데이터 경로 지정
dic = {
'train' : '/content/yolov5/data/images/train.txt',
'val' : '/content/yolov5/data/images/valid.txt',

## 클래스 수
'nc' : 3 ,

## 종류
'names' : ['Zebra_Cross', 'G_Signal', 'R_Signal']
}

with open('/content/yolov5/data/dataset.yaml', 'w') as f :
    yaml.dump(dic, f)


## yolo v5 모델 가서 nc=으로 바꿔주기!




In [ ]:
%cd /content/yolov5

/content/yolov5


# Training

In [ ]:
!python train.py --img 640 --batch 32 --epochs 10 --data ./data/dataset.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name yolo_test_first

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 2
wandb: You chose 'Use an existing W&B account'
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=./data/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolo_test_first, exist_ok=False, quad=False, cos

In [ ]:
!zip -r /content/yolov5/runs.zip /content/yolov5/runs/

updating: content/yolov5/runs/ (stored 0%)
updating: content/yolov5/runs/train/ (stored 0%)
updating: content/yolov5/runs/train/yolo_test_first/ (stored 0%)
updating: content/yolov5/runs/train/yolo_test_first/weights/ (stored 0%)
updating: content/yolov5/runs/train/yolo_test_first/weights/best.pt (deflated 9%)
updating: content/yolov5/runs/train/yolo_test_first/weights/last.pt (deflated 9%)
updating: content/yolov5/runs/train/yolo_test_first/labels.jpg (deflated 21%)
updating: content/yolov5/runs/train/yolo_test_first/opt.yaml (deflated 44%)
updating: content/yolov5/runs/train/yolo_test_first/results.png (deflated 9%)
updating: content/yolov5/runs/train/yolo_test_first/val_batch2_labels.jpg (deflated 5%)
updating: content/yolov5/runs/train/yolo_test_first/val_batch0_labels.jpg (deflated 5%)
updating: content/yolov5/runs/train/yolo_test_first/results.csv (deflated 80%)
updating: content/yolov5/runs/train/yolo_test_first/hyp.yaml (deflated 45%)
updating: content/yolov5/runs/train/yolo_te

# Inference YouTube Video

In [ ]:

!python detect.py --weights /content/yolov5/runs/train/yolo_test_first/weights/best.pt --img 640 --conf 0.5 --source "https://www.youtube.com/watch?v=VVl6chJ_xIo"


detect: weights=['/content/yolov5/runs/train/yolo_test_first/weights/best.pt'], source=https://www.youtube.com/watch?v=VVl6chJ_xIo, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-277-gfdc9d91 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
cv2.imshow() is disabled in Google Colab environments
requirements: pafy not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: youtube_dl==2020.12.2 not found and is requir

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/yolo_test_first/weights/best.pt --img 640 --conf 0.5 --source "https://www.youtube.com/watch?v=p6fN5swYYL8"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.007s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.008s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.007s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.008s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.010s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0: 384x640 Done. (0.006s)
0:

In [ ]:
from google.colab import files
files.download('/content/yolov5/runs/detect/exp3/watch_v_p6fN5swYYL8.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>